In [ ]:
#@markdown # Use Google Drive (optional)
#@markdown Models will be saved and loaded from your Google Drive.
from google.colab import drive
mount_google_drive = True
drive.mount('/content/gdrive')

In [ ]:
#@markdown # Download repo
update_repo = False #@param {type:"boolean"}

if not "mount_google_drive" in locals():
  mount_google_drive = False

if mount_google_drive:
  %cd /content/gdrive/MyDrive/
else:
  %cd /content/

if update_repo:
  %cd nataili/
  !git pull
else:
  !git clone https://github.com/Sygil-Dev/nataili.git

In [ ]:
#@markdown # Install Python 3.8
!sudo apt-get update -y
!sudo apt-get install python3.8

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2

#check python version
!python --version

# Install pip
!sudo apt install python3-pip
!python -m pip install --upgrade pip

In [ ]:
#@markdown # Install dependencies

if not "mount_google_drive" in locals():
  mount_google_drive = False

if mount_google_drive:
  %cd /content/gdrive/MyDrive/nataili/
else:
  %cd /content/nataili

!pip install -e .

!pip install ray

# See: https://github.com/CompVis/taming-transformers/issues/176
# do not uncomment -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install invisible-watermark==0.1.5
!pip install taming-transformers-rom1504==0.0.6  # required by ldm

!pip install git+https://github.com/crowsonkb/k-diffusion.git

# Dependencies required for Stable Diffusion UI
!pip install pynvml==11.4.1
!pip install omegaconf==2.2.3

# Note: Jinja2 3.x major version required due to breaking changes found in markupsafe==2.1.1; 2.0.1 is incompatible with other upstream dependencies
# see https://github.com/pallets/markupsafe/issues/304
!pip install Jinja2==3.1.2  # Jinja2 is required by Gradio

#!pip install diffusers==0.4.1
!pip install diffusers==0.6.0

# Img2text
!pip install fairscale==0.4.4
!pip install timm==0.6.7
!pip install tqdm==4.64.0

# Other
!pip install retry==0.9.2  # used by sd_utils
!pip install python-slugify==6.1.2  # used by sd_utils
!pip install piexif==1.1.3  # used by sd_utils

!pip install accelerate==0.12.0
!pip install albumentations==0.4.3
!pip install einops==0.3.1
!pip install facexlib>=0.2.3
!pip install imageio-ffmpeg==0.4.2
!pip install imageio==2.9.0
!pip install kornia==0.6
!pip install loguru
!pip install opencv-python-headless==4.6.0.66
!pip install open-clip-torch==2.0.2
!pip install pandas==1.4.3
!pip install pudb==2019.2
!pip install pytorch-lightning==1.7.7
!pip install realesrgan==0.3.0
!pip install test-tube>=0.7.5
!pip install timm==0.6.7
!pip install torch-fidelity==0.3.0
#do not change transformers version
!pip install transformers==4.21.3 # do not change
#do not change transformers version
!pip install wget

# Upscalers
!pip install basicsr==1.4.2  # required by RealESRGAN
!pip install gfpgan==1.3.8  # GFPGAN
!pip install realesrgan==0.3.0  # RealESRGAN brings in GFPGAN as a requirement
!pip install git+https://github.com/CompVis/latent-diffusion

## for monocular depth estimation 
!pip install tensorflow==2.10.0

In [ ]:
#@markdown # Install xformers
!wget https://github.com/aqualxx/stable-horde-notebook/raw/main/precompiled/T4/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl -O ./xformers-0.0.14.dev0-py3-none-any.whl
!pip install xformers-0.0.14.dev0-py3-none-any.whl

# Setting worker info
1. Rename `bridgeData_template.py` to `bridgeData.py`. 
2. Edit `bridgeData.py` and enter details such as the `horde_name` and `horde_api_key` you've received, so that you can receive Kudos.
3. If you downloaded extra models other than 1.5, list it in `models_to_load`.

OR

1. Fill and run the cells bellow.

**Note**: If you do not do this step, you will contribute anonymously.

In [ ]:
#@markdown # List available models (for models_to_load)
if not "mount_google_drive" in locals():
  mount_google_drive = False

if mount_google_drive:
  %cd /content/gdrive/MyDrive/nataili/
else:
  %cd /content/nataili

import requests, json

models = json.load(open("./db.json"))

try:
    r = requests.get("https://raw.githubusercontent.com/Sygil-Dev/nataili-model-reference/main/db.json")
    models = r.json()
except Exception:
    models = json.load(open("./db.json"))

filtered_models = {}
for model_name in models:
  if models[model_name].get("type") == "ckpt":
    filtered_models[model_name] = models[model_name]

ppmodels = ""
for model_name in filtered_models:
    if model_name == "LDSR":
        continue
    ppmodels += model_name
    if filtered_models[model_name].get("description"):
        ppmodels += f" : {filtered_models[model_name].get('description')}"
    ppmodels += "\n"
print(f"## Known ckpt Models ##\n{ppmodels}")

In [ ]:
# The horde url
horde_url = "https://stablehorde.net"
# Give a cool name to your instance
worker_name = "My Awesome Instance"
# The api_key identifies a unique user in the horde
# Visit https://stablehorde.net/register to create one before you can join
api_key = "0000000000"
# Put other users whose prompts you want to prioritize.
# The owner's username is always included so you don't need to add it here,
# unless you want it to have lower priority than another user
priority_usernames = []
# The amount of power your system can handle
# 8 means 512*512. Each increase increases the possible resoluion by 64 pixes
# So if you put this to 2 (the minimum, your SD can only generate 64x64 pixels
# If you put this to 32, it is equivalent to 1024x1024 pixels
max_power = 8
# The amount of parallel jobs to pick up for the horde. Each running job will consume the amount of RAM needed to run each model, and will also affect the speed of other running jobs
# so make sure you have enough VRAM to load models in parallel, and that the speed of fulfilling requests is not too slow
# Expected limit per VRAM size: <6 VMAM: 1, <=8 VRAM: 2, <=12 VRAM:3, <=14 VRAM: 4
# But remember that the speed of your gens will also be affected for each parallel job
max_threads = 1
# Set this to false, if you do not want your worker to receive requests for NSFW generations
nsfw = True
# Set this to True if you want your worker to censor NSFW generations. This will only be active is horde_nsfw == False
censor_nsfw = False
# A list of words which you do not want to your worker to accept
blacklist = []
# A list of words for which you always want to allow the NSFW censor filter, even when this worker is in NSFW mode
censorlist = []
# If set to False, this worker will no longer pick img2img jobs
allow_img2img = True
# If set to True, this worker will can pick inpainting jobs
allow_painting = True
# If set to False, this worker will no longer pick img2img jobs from unsafe IPs
allow_unsafe_ip = True

# If you set this to True, the worker will detect the most popular models and load them automatically ( Defaults to True if missing )
# Note this ultimately overrides the models_to_load list
dynamic_models = True

# Adjust how many models to load into memory. In future this will likely be an argument for memory size or may disappear, but for right now, I'm lazy
number_of_dynamic_models = 3
# The models to use. You can select a different main model, or select more than one.
# With you can easily load 20 of these models with 32Gb RAM and 6G VRAM.
# Adjust how many models you load based on how much RAM (not VRAM) you have available.
# The last model in this list takes priority when the client accepts more than 1
# if you do not know which models you can add here, use the below command
# python show_available_models.py
## WARNING: In case you have dynamic models this list is instead specifying models to always load!
# in that case, keep this list short, preferrably to only a few more obscure models you'd like to always see available
# and at most at half the size of your number_of_dynamic_models number
models_to_load = [
    ## Generalist:
    "stable_diffusion",  # This is the standard compvis model. It is not using Diffusers (yet)
    # "stable_diffusion_2.0", # Dont use this yet. It is not implemented and will fail 
    # "Midjourney Diffusion", # Similar to Midjourney v4 images generations

    ## Enable this to allow inpainting/outpainting.
    # "stable_diffusion_inpainting",

    ## Generalist-merged:
    # "Mega Merge Diffusion", # SD 1.5 merged with 17 other models 
    # "Nitro Diffusion", # Multi-Style model trained on Arcane, Archer and Mo-Di 
    # "Samdoesarts Ultmerge", # Portraits in the style of Sam Yang, merged with chewtoy and orange code-s models 

    ## 3D
    # "Clazy", # Generates clay-like figures 
    # "Microworlds", # Isometric microworlds 
    # "Papercut Diffusion", # Paper cut images 
    # "Redshift Diffusion", # high resolution 3D artworks 
    # "Voxel Art Diffusion", # Voxel art style generations 

    ## Animation-Anime-Cartoon-Manga:
    # "Anything Diffusion", # Highly detailed Anime styled generations 
    # "Classic Animation Diffusion", # Popular animation studio classic style generations 
    # "Comic-Diffusion", # Western Comic book style 
    # "Cyberpunk Anime Diffusion", # Cyberpunk anime characters 
    # "Furry Epoch", # Furry styled generations 
    # "Ghibli Diffusion", # Studio Ghibli feature films style 
    # "Hentai Diffusion", # NSFW: Anime focused model with better hands, obscure poses/camera angles and consistent style 
    # "mo-di-diffusion", # Popular animation studio modern style generations 
    # "Ranma Diffusion", # Imitates the style of late 80s early 90-s anime, Anything v3 base 
    # "Spider-Verse Diffusion", # Based on the Into the Spider-Verse movie's animation style 
    # "trinart", # Manga styled generations 
    # "Yiffy", # Furry styled generations 
    # "Zack3D", # NSFW: Kink oriented furry styled generations 
    # "waifu_diffusion", # Anime styled generations 

    ## Fandom styles:
    # "Arcane Diffusion", # Arcane TV show 
    # "Archer Diffusion", # Archer-s TV show animation style 
    # "Borderlands", # Borderlands video game style 
    # "Dungeons and Diffusion", # Generates DnD styled characters, trained on art commissions 
    # "Elden Ring Diffusion", # Based on the Elden Ring video game style 
    # "Tron Legacy Diffusion", # Tron Legacy movie style 
    # "Xynthii-Diffusion", # Xynthii: cyclops monster girls  

    ## Painting styles
    # "AIO Pixel Art", #Stable Diffusion fine tuned on pixel art sprites and scenes 
    # "BubblyDubbly", # Dreamy sketched-painted portraits 
    # "colorbook", # Minimalist coloring book style generations 
    # "Guohua Diffusion", # Traditional Chinese paintings generations 
    # "Van Gogh Diffusion", # film Loving Vincent, best results with k_euler sampler 
    # "vectorartz", # Generate beautiful vector illustration 

    ## Others
    # "App Icon Diffusion", # Mobile app icons 
    # "JWST Deep Space Diffusion", # James Webb Space Telescope imagery 
    # "Robo-Diffusion", # Robot oriented drawing style 
    # "Synthwave", # Synthwave-outrun style 
    # "Zeipher Female Model", # NSFW: For creating images of nude solo women. Also known as f222 
]

# This is used when dynamic_models == True
# The models in this list will not be loaded when they exist in the top models
# This is to avoid loading models which you do not want either due to VRAM constraints, or due to NSFW content
models_to_skip = [
    "stable_diffusion_inpainting",  # Inpainting is generally quite heavy along with other models for smaller GPUs.
    "stable_diffusion_2.1",  # Stable diffusion 2.1 has bigger memory requirements than 1.5, so if your card cannot lift, it, disable it
    "stable_diffusion_2.0",  # Same as Stable diffusion 2.1
]

In [ ]:
#@markdown # Start Stable Horde worker
import os

huggingface_token = "" #@param {type:"string"}

if not "mount_google_drive" in locals():
  mount_google_drive = False

if mount_google_drive:
  %cd /content/gdrive/MyDrive/nataili/
else:
  %cd /content/nataili/

if "api_key" in locals():
  data = f"""horde_url = "{horde_url}"
worker_name = "{worker_name}"
api_key = "{api_key}"
priority_usernames = {priority_usernames}
max_power = {max_power}
max_threads = {max_threads}
nsfw = {nsfw}
censor_nsfw = {censor_nsfw}
blacklist = {blacklist}
censorlist = {censorlist}
allow_img2img = {allow_img2img}
allow_painting = {allow_painting}
allow_unsafe_ip = {allow_unsafe_ip}
dynamic_models = {dynamic_models}
number_of_dynamic_models = {number_of_dynamic_models}
models_to_load = {models_to_load}
models_to_skip = {models_to_skip}"""
  toExec = """text_file = open("bridgeData.py", "w+"); text_file.write(data); text_file.close()""" 
  exec(toExec)

!python bridge.py -vvvvvv --disable_voodoo --hf_token $huggingface_token